# Sementic segmentation of flair1 data with a basic UNet

In [4]:
# import all the necessary modules
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path
from utils import *
import numpy as np
from sklearn.model_selection import train_test_split
import torch.utils.data as data

/home/bertille/miniconda3/envs/env_pytorch/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare the data
1) Download the data and split it into training, validation and test sets
2) Create a Dataloader
3) Load the data

In [3]:
folder_path = '../../data/flair1/'
dict_classes = {
1   : 'building',
2   : 'pervious surface',
3   : 'impervious surface',
4   : 'bare soil',
5   : 'water',
6   : 'coniferous',
7   : 'deciduous',
8   : 'brushwood',
9   : 'vineyard',
10  : 'herbaceous vegetation',
11  : 'agricultural land',
12  : 'plowed land',
13  : 'swimming_pool',
14  : 'snow',
15  : 'clear cut',
16  : 'mixed',
17  : 'ligneous',
18  : 'greenhouse',
19  : 'other'}

colors = {
1   : '#db0e9a',
2   : '#938e7b',
3   : '#f80c00',
4   : '#a97101',
5   : '#1553ae',
6   : '#194a26',
7   : '#46e483',
8   : '#f3a60d',
9   : '#660082',
10  : '#55ff00',
11  : '#fff30d',
12  : '#e4df7c',
13  : '#3de6eb',
14  : '#ffffff',
15  : '#8ab3a0',
16  : '#6b714f',
17  : '#c5dc42',
18  : '#9999ff',
19  : '#000000'}

images = sorted(list(get_data_paths(Path(folder_path), 'IMG*.tif')), key=lambda x: int(x.split('_')[-1][:-4]))[0:100]
masks  = sorted(list(get_data_paths(Path(folder_path), 'MSK*.tif')), key=lambda x: int(x.split('_')[-1][:-4]))[0:100]

#split into train, validation and test
images_train, images_test = train_test_split(images, test_size=0.2, random_state=42)
masks_train, masks_test = train_test_split(masks, test_size=0.2, random_state=42)
images_train, images_val = train_test_split(images_train, test_size=0.2, random_state=42)
masks_train, masks_val = train_test_split(masks_train, test_size=0.2, random_state=42)

In [5]:
class DataLoaderSegmentation(data.Dataset):
    def __init__(self, folder_path):
        super(DataLoaderSegmentation, self).__init__()
        self.img_files = sorted(list(get_data_paths(Path(folder_path), 'IMG*.tif')), key=lambda x: int(x.split('_')[-1][:-4]))
        self.mask_files = sorted(list(get_data_paths(Path(folder_path), 'MSK*.tif')), key=lambda x: int(x.split('_')[-1][:-4]))

    def __getitem__(self, index):
        img_path = self.img_files[index]
        mask_path = self.mask_files[index]
        data = rasterio.open(img_path).read()
        label = rasterio.open(mask_path).read()
        return torch.from_numpy(data).float(), torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)

In [ ]:
train_loader = DataLoaderSegmentation(folder_path)

## Create the model

## Create the functions for training, validation and test, with plots of the loss

## Run the algorithm for a tiny subset